In [7]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

def is_same_domain(url, base_domain):
    """URLが同じドメインかどうかを確認する関数"""
    parsed_url = urlparse(url)
    return parsed_url.netloc == base_domain

def extract_sitemap(start_url):
    """
    ウェブサイトをクロールしてサイトマップを抽出する関数
    戻り値: {URL: タイトル} の辞書
    """
    base_domain = urlparse(start_url).netloc
    visited_urls = set()  # 訪問済みURLを記録
    to_visit = [start_url]  # 訪問予定のURL
    url_title_dict = {}  # {URL: タイトル} の辞書
    
    while to_visit:
        current_url = to_visit.pop(0)
        
        # 既に訪問済みのURLはスキップ
        if current_url in visited_urls:
            continue
        
        # 訪問済みに追加
        visited_urls.add(current_url)
        
        try:
            # サーバー負荷軽減のため一定時間待機
            time.sleep(0.5)
            
            print(f"アクセス中: {current_url}")
            response = requests.get(current_url, timeout=10)
            
            # HTTPステータスコードが200以外の場合はスキップ
            if response.status_code != 200:
                print(f"アクセス失敗: {current_url}, ステータスコード: {response.status_code}")
                continue
            
            # HTMLの解析
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # タイトルの取得
            title_tag = soup.find('title')
            title = title_tag.string.strip() if title_tag else "タイトルなし"
            
            # URL:タイトルの辞書に追加
            url_title_dict[current_url] = title
            
            # 同じドメインの全てのリンクを抽出
            for link in soup.find_all('a', href=True):
                href = link['href']
                
                # 相対URLを絶対URLに変換
                full_url = urljoin(current_url, href)
                
                # フラグメント(#以降)を削除
                full_url = full_url.split('#')[0]
                
                # 同一ドメインかつ未訪問のURLのみを追加
                if is_same_domain(full_url, base_domain) and full_url not in visited_urls and full_url not in to_visit:
                    # JavaScriptのリンクやメールアドレスは除外
                    if not href.startswith(('javascript:', 'mailto:')):
                        to_visit.append(full_url)
        
        except Exception as e:
            print(f"エラー発生: {current_url}, {str(e)}")
    
    return url_title_dict

# 武蔵野大学のWebサイトURLを設定
base_url = "https://www.musashino-u.ac.jp"

# サイトマップの抽出
site_map = extract_sitemap(base_url)

# 結果の表示
print("\n--- 武蔵野大学WebサイトのサイトマップTOP100 ---")
count = 0
for url, title in site_map.items():
    print(f"{url}: {title}")
    count += 1
    if count >= 100:
        print(f"\n※全{len(site_map)}ページ中100ページのみ表示")
        break

print(f"\n合計: {len(site_map)}ページ")

アクセス中: https://www.musashino-u.ac.jp
アクセス中: https://www.musashino-u.ac.jp/
アクセス中: https://www.musashino-u.ac.jp/access.html
アクセス中: https://www.musashino-u.ac.jp/admission/request.html
アクセス中: https://www.musashino-u.ac.jp/contact.html
アクセス中: https://www.musashino-u.ac.jp/prospective-students.html
アクセス中: https://www.musashino-u.ac.jp/students.html
アクセス中: https://www.musashino-u.ac.jp/alumni.html
アクセス中: https://www.musashino-u.ac.jp/parents.html
アクセス中: https://www.musashino-u.ac.jp/business.html
アクセス中: https://www.musashino-u.ac.jp/guide/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/
アクセス中: https://www.musashino-u.ac.jp/guide/activities/
アクセス中: https://www.musashino-u.ac.jp/guide/campus/
アクセス中: https://www.musashino-u.ac.jp/guide/facility/
アクセス中: https://www.musashino-u.ac.jp/guide/information/
アクセス中: https://www.musashino-u.ac.jp/guide/profile/media/
アクセス中: https://www.musashino-u.ac.jp/admission/
アクセス中: https://www.musashino-u.ac.jp/admission/faculty/
アクセス中: https://www.musashino-